<h2>
    <p>
        MÔN BIG DATA 2019
    </p>
    <p style="color:red">
        Nhóm 13
    </p>
</h2>

<h3>
    <p>
        SỬ DỤNG SPARK VÀ SPARK MACHINE LEARNING LIBRARY ĐỂ TRAIN MODEL PHÂN TÍCH NGỮ NGHĨA DÙNG MULTILAYER PERCEPTRONS (BASIC DEEP LEARNING)
    </p>
    <p style="color: blue">
        Phần áp dụng Model đã train vào phân tích dữ liệu Real Time sử dụng Kafka
    </p>
     <img src="https://lh3.googleusercontent.com/bp9_jpWQssGTk3VzLFlywR8jDNlyGWYc-9vIDTb1-1ovsTQzfMPhTbwrZv12n1YAAbtW4s5eYc-pxOXgfYyX3w7cseef_IsmVAjc_j2VRvv3aCrKQFSdA_v3YpTQyxwBtPgh0IpJWMFe9boJ2tULIzlAT18bXCJN2IGQ-gXZkfEG0_HtV6-oN4l7qahMhZpCcE4N8AX048YRWWyLZAPFprvtRiebkJHRv5aLk1zNqkZP3fxTu67ltR148Tm2NoEl6OS_DanNwgHEHVbI3aXmoy0uVlI6d6t1sTtcq7oGuNfmXM04Eftu_VKe46wBsBrQ0lSLn6JPnzU0t0krEGwJ5fbDMD1-qo0wStiz4i2Tr9ivgUVRSwqPBXW-WtKli6qcZW-PfDJg2Gga84qL2pZ5uEG5wjLuP0_FqcwC_Ngt-e2nwXCa7QQ-Uxh7oSD3zpu2unlIHfufExxnOc5Ub8CdlhLRmnQdOVE-R6rMon1s8iwNBvr6yTaOCXkaxltzomnDh0Y09G3sGTMTz_FHkjBxdjv3STRAt7T4QmqxvA9HyyzoTGZhGhMF4sCfLU7PM-HmcLQGCgWdAOTaIgBmfJw099QWckc7wnqHFR8KAHC1bIFDn2yCPomnZjO2cpmL4K1KmQGlzvTM40f1ynw7DhzF6KoT_ucbFn2t6KUOLjLAACTLdoKCmSG4cUU=w1885-h899-no" alt="" height="100%" width="100%"> 
</h3>

<h4>
    <p style="color:blue">
        Khai báo các biến môi trường để Spark sử dụng trong quá trình chạy
    </p>
</h4>

In [18]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/home/spark/spark-2.4.3-bin-hadoop2.7"
#os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local --packages org.apache.spark:spark-streaming-kafka-0-8:2.3.0 pyspark-shell"

In [19]:
import findspark
findspark.init()

<h4>
    <p style="color:red">
        Import các thư viện của PySpark
    </p>
</h4>

In [20]:
from os import listdir
from pyspark import SparkContext
from pyspark.ml import PipelineModel
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import *

<h4>
    <p style="color:blue">
        Khai báo số memory mà mỗi Worker sẽ sử dụng để thực thi task
    </p>
</h4>

In [21]:
SparkContext.setSystemProperty('spark.executor.memory', '6656m')

<h4>
    <p style="color:red">
        Khai báo địa chỉ của Spark master, lưu ý là phải thay đổi địa chỉ cho phù hợp với từng Spark cluster
    </p>
</h4>

In [22]:
# tạo spark context, lưu ý nhớ thay đổi lại địa chỉ của spark cho phù hợp
sc = SparkContext("spark://10.255.255.6:7077", "Kafka Semantic Handle Real Time")
spark = SparkSession.builder.getOrCreate()

<h4>
    <p style="color:red">
      Sử dụng lại model (Load Model) đã lưu sử dụng lệnh dưới đây
    </p>
</h4>

In [23]:
pl_model = PipelineModel.load("hdfs://s14e18f4e58324b66b78ecd8c831a0c6413-master.uitlab.com:9000/sparkdata/pl2.model")
pl_model

PipelineModel_49b6081081c8

<h4>
    <p style="color:blue">
        Import các thư viện của PySpark Streaming và Kafka sử dụng trong Spark
    </p>
</h4>

In [24]:
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from pyspark.sql.types import *

<h4>
    <p style="color:red">
        Khai báo địa chỉ của Kafka Cluster và topic để lấy dữ liệu
    </p>
</h4>

In [25]:
kafkaBroker = 'localhost:9092'
topic = 'spark-sematic-input1'
batchTimeDur = 1

<h4>
    <p style="color:blue">
        Tạo đối tượng Streaming, với Time là 1s sẽ fetch dữ liệu một lần
    </p>
</h4>

In [26]:
ssc = StreamingContext(sc, batchTimeDur)

<h4>
    <p style="color:red">
        Tạo luồng Streaming Kafka ứng với Spark Context và Topic đã khai báo
    </p>
</h4>

In [27]:
directKafkaStream = KafkaUtils.createDirectStream(ssc, [topic], {"metadata.broker.list": kafkaBroker})

<h4>
    <p>
        Import thư viện xử lý Kafka
    </p>
</h4>

In [28]:
from kafka import KafkaProducer
import json

<h4>
    <p style="color:blue">
        Tạo Kafka Producer để đẩy dữ liệu sau khi đã đi qua Model Deep Learning quay ngược trở về Kafka, cung cấp dữ liệu cho các nguồn khác.
    </p>
</h4>

In [29]:
producer = KafkaProducer(bootstrap_servers='localhost:9092')

<h4>
    <p style="color:red">
        Function để xử lý dữ liệu trong Streaming
    </p>
</h4>

In [30]:
def handler(message):
    records = message.collect()
    data = []
    index = 1
    for record in records:
        key, value = record
        #print(value)
        data.append((index, value))
        index = index + 1
    if len(data) > 0:
        #print(data)
        sentenceDataFrame = spark.createDataFrame(data, ["id", "sentence"])
        new_dt = pl_model.transform(sentenceDataFrame)
        new_dt = new_dt.select("sentence", "prediction").collect()
        
        for row in new_dt:
            s = row['sentence']
            p = row['prediction']
            
            o = {
                'sentence': s,
                'prediction': p
            }
            
            j = json.dumps(o)
            
            print(j)
            
            j = str.encode(j)
            
            producer.send("spark-sematic-output1", j)
            producer.flush()
            
            #print(j)
            
        
        #print(new_dt)

<h4>
    <p>
        Đăng ký funtion cho luồng Kafka Streaming
    </p>
</h4>

In [31]:
directKafkaStream.foreachRDD(handler)

<h4>
    <p style="color:blue">
        Start Streaming và xem kết quả.
    </p>
    <p style="color:red">
        Mỗi khi có luồng dữ liệu đến, data sẽ chạy qua model để được giá trị prediction.
    </p>
    <p style="color:red">
        Sau khi có kết quả, data kết quả sẽ được submit lên một topic Kafka mới, Topic output này sẽ là nguồn dữ liệu cho việc lưu trữ xuống MongoDB hoặc được hiển thị lên biểu đồ dạng Real Time.
    </p>
</h4>

In [ ]:
ssc.start()
ssc.awaitTermination()

{"prediction": 1.0, "sentence": "This is a pale imitation of 'Officer and a Gentleman.' There is NO chemistry between Kutcher and the unknown woman who plays his love interest. The dialog is wooden, the situations hackneyed. It's too long and the climax is anti-climactic(!). I love the USCG, its men and women are fearless and tough. The action scenes are awesome, but this movie doesn't do much for recruiting, I fear. The script is formulaic, but confusing. Kutcher's character is trying to redeem himself for an accident that wasn't his fault? Costner's is raging against the dying of the light, but why? His 'conflict' with his wife is about as deep as a mud puddle. I saw this sneak preview for free and certainly felt I got my money's worth."}


<h4>
    <p>
        Kết Thúc một phiên làm việc trong Spark
    </p>
</h4>

In [14]:
print("end_of_code")
spark.stop()

end_of_code
